In [3]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Deep Learning API for creating Neural Networks (Runs on TensorFlow)
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import math
from keras.utils.vis_utils import plot_model

from sklearn.metrics import accuracy_score

from my_utils.visualization import show_history_graph, show_ensemble_features

In [2]:
# os.chdir('c:\\PROJEKTY\\KAGGLE\\SpaceshipTytanic\\')

# 0 - Data

In [4]:
train_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\train_processed.ftr')
val_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\val_processed.ftr')
cols = list(train_df.columns)

In [5]:
x_train = train_df[cols[:-1]].to_numpy()
x_val = val_df[cols[:-1]].to_numpy()
y_train = train_df[cols[-1]].astype(float).to_numpy()
y_val = val_df[cols[-1]].astype(float).to_numpy()

# 1 - Baseline model
* Train and val accuracy are almost the same (~0.78)
* It means that the model in under-fitted

In [6]:
def get_accuracy(x, y, name, model):
    y_pred = model.predict(x)
    y_pred = [1 if y>=0.5 else 0 for y in y_pred.reshape(1,-1)[0]]
    acc = accuracy_score(y, y_pred)
    print(f'Accuracy on {name} data: {round(acc, 3)}')

In [11]:
layers = [
  Dense(4, activation=keras.activations.relu), # hidden layer 1, ReLU activation
  Dense(4, activation=keras.activations.relu), # hidden layer 2, ReLU activation
  Dense(1, activation=keras.activations.sigmoid) # ouput layer, sigmoid activation
]

base_model = keras.Sequential(layers)

In [12]:
base_model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [13]:
history = base_model.fit(
    x_train,
    y_train,
    epochs=100,
    validation_data=(x_val, y_val)
)

Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7642 - accuracy: 0.4901 - val_loss: 0.6502 - val_accuracy: 0.6554
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5978 - accuracy: 0.7185 - val_loss: 0.5719 - val_accuracy: 0.7270
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5475 - accuracy: 0.7452 - val_loss: 0.5460 - val_accuracy: 0.7400
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5248 - accuracy: 0.7549 - val_loss: 0.5231 - val_accuracy: 0.7577
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5092 - accuracy: 0.7683 - val_loss: 0.5103 - val_accuracy: 0.7683
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5007 - accuracy: 0.7709 - val_loss: 0.5001 - val_accuracy: 0.7742
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4955 - accuracy: 0.7717 - val_loss: 0.4960 - val_accuracy: 0.7736

In [15]:
get_accuracy(x_train, y_train, 'train', base_model)

219/219 [==============================] - 0s 1ms/step
Accuracy on train data: 0.78


In [16]:
get_accuracy(x_val, y_val, 'train', base_model)

53/53 [==============================] - 0s 1ms/step
Accuracy on train data: 0.788


In [14]:
history_df = pd.DataFrame(history.history)

show_history_graph(history_df, 'loss', 'val_loss')
show_history_graph(history_df, 'accuracy', 'val_accuracy')

# 2 - More layers, more neurons, more dropout!
* More layers with more neurons and Dropout actions got better model
* Both train and val accuracy is near ~0.80

In [44]:
relu = keras.activations.relu
layers = [
    Dense(512, activation=relu),
    Dropout(0.4),
    Dense(256, activation=relu),
    Dropout(0.4),
    Dense(128, activation=relu),
    Dropout(0.3),
    Dense(64, activation=relu),
    Dropout(0.2),
    Dense(32, activation=relu),
    Dropout(0.15),
    Dense(16, activation=relu),
    Dropout(0.1),
    Dense(8, activation=relu),
    Dense(4, activation=relu),
    Dense(1, activation=keras.activations.sigmoid)
]

model_2 = keras.Sequential(layers)

In [45]:
model_2.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=20,
    verbose=0,
    restore_best_weights=True
)
callbacks = [
    early_stopping_callback
]

In [46]:
history = model_2.fit(
    x_train,
    y_train,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks
)

Epoch 1/100
219/219 [==============================] - 2s 5ms/step - loss: 0.6052 - accuracy: 0.7310 - val_loss: 0.5804 - val_accuracy: 0.7660
Epoch 2/100
219/219 [==============================] - 1s 4ms/step - loss: 0.5690 - accuracy: 0.7723 - val_loss: 0.5476 - val_accuracy: 0.7884
Epoch 3/100
219/219 [==============================] - 1s 5ms/step - loss: 0.5447 - accuracy: 0.7813 - val_loss: 0.5335 - val_accuracy: 0.7831
Epoch 4/100
219/219 [==============================] - 1s 5ms/step - loss: 0.5367 - accuracy: 0.7833 - val_loss: 0.5244 - val_accuracy: 0.7825
Epoch 5/100
219/219 [==============================] - 1s 6ms/step - loss: 0.5256 - accuracy: 0.7797 - val_loss: 0.5179 - val_accuracy: 0.7937
Epoch 6/100
219/219 [==============================] - 2s 7ms/step - loss: 0.5198 - accuracy: 0.7826 - val_loss: 0.5129 - val_accuracy: 0.7819
Epoch 7/100
219/219 [==============================] - 2s 8ms/step - loss: 0.5137 - accuracy: 0.7846 - val_loss: 0.5099 - val_accuracy: 0.7866

In [49]:
get_accuracy(x_train, y_train, 'train', model_2)

219/219 [==============================] - 0s 2ms/step
Accuracy on train data: 0.8


In [50]:
get_accuracy(x_val, y_val, 'val', model_2)

53/53 [==============================] - 0s 2ms/step
Accuracy on val data: 0.798


In [22]:
history_df = pd.DataFrame(history.history)

show_history_graph(history_df, 'loss', 'val_loss')
show_history_graph(history_df, 'accuracy', 'val_accuracy')

# 3 - Add new ensembe feature

In [52]:
train_ensemble_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_train_df.ftr')
val_ensemble_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_val_df.ftr')

In [54]:
features = ['neural', 'lgbm', 'ada_boost', 'svc']

train_ensemble_feature = model_2.predict(x_train)
train_ensemble_df['neural'] = train_ensemble_feature
train_ensemble_df['mean'] = train_ensemble_df[features].mean(axis=1)

val_ensemble_feature = model_2.predict(x_val)
val_ensemble_df['neural'] = val_ensemble_feature
val_ensemble_df['mean'] = val_ensemble_df[features].mean(axis=1)

53/53 [==============================] - 0s 1ms/step


In [55]:
cols = ['neural', 'lgbm', 'ada_boost', 'svc', 'mean', 'y']
train_ensemble_df[cols].to_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_train_df.ftr')
val_ensemble_df[cols].to_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_val_df.ftr')

In [56]:
features = ['neural', 'lgbm', 'ada_boost', 'svc', 'mean']
show_ensemble_features(train_ensemble_df, features, 'train')